In [ ]:
import importlib

import os
import sys
import json
import warnings
import traceback

#from scipy import signal
import numpy as np
from tqdm import tqdm

sys.path.insert(0, '/mnt/d/projects/bassline_extraction') 

from plotting.various import energy_levels
import transcription

from signal_processing import *
from chorus_estimation import *
from utilities import *

warnings.filterwarnings('ignore')
#np.set_printoptions(suppress=True)

# Initialization

## Directories

**TO DO: aligned_beat_positions => progression_beat_positions**

In [ ]:
with open('../data/ouz_tracks.txt', 'r') as infile:
    track_titles = infile.read().split('\n')
       
_,_, bad_examples = next(os.walk('../data/bassline_extraction_FIRST_EXPERIMENT/beat_grid/bad_examples'))
bad_examples = [title.split('.txt')[0] for title in bad_examples]
       
with open('../data/metadata/scales_frequencies.json','r') as infile:
    scales = json.load(infile)

with open('../data/metadata/TechHouse_track_dicts.json','r') as infile:
    track_dicts = json.load(infile) 

# Listening and Waveform Inspection

Select a track to work with

In [ ]:
search_idx("Bontan - State Of Mind (Original Mix) [Suara]", track_titles)

In [ ]:
#title = track_titles[np.random.randint(len(track_titles))] 
title = track_titles[27]

print('{}'.format(title))
BPM = float(track_dicts[title]['BPM'])
print('BPM: {}\n'.format(BPM))
beat_length = 60/BPM

if title in bad_examples:
    with open(os.path.join('../data/bassline_extraction_FIRST_EXPERIMENT/beat_grid/bad_examples',title+'.txt'), 'r') as infile:
        dpqb = float(infile.read())
    print("Beat grid problematic! dpqb: {:.2f}%\n{}".format(dpqb,title))
    
beat_positions = get_beat_positions(title)
bar_positions = get_bar_positions(beat_positions)
    
fs = 44100

# Energy Calculations

**Load the track, LP filter it and calculate the bar and cell energies**

In [ ]:
track = load_track(title, fs)[0]

track_lp = lp_and_normalize(256, fs, 1001, track)

bar_energies = calculate_bar_energies(track_lp, bar_positions, fs)

cell_energies = calculate_cell_energies(bar_energies)
smoothed_cell_energies = sample_and_hold(cell_energies, 4)

**Detect Choruses**

In [ ]:
# directly detect drop

#eps = 2
#estimated_drop = drop_detection(smoothed_cell_energies, eps, 1000)

In [ ]:
eps = 2

possible_drops = find_drops(smoothed_cell_energies, epsilon=eps)
estimated_drop = drop_picking(possible_drops)

drop_beat_idx = estimated_drop[0]*4

chorus_beat_positions = beat_positions[drop_beat_idx:drop_beat_idx+16]

**Check Chorus Beat Grid**

In [ ]:
beat_factor = 64

tolerance = beat_length/beat_factor

rounded_beat_length = np.around(beat_length, 2)
#rounded_tolerance = np.around(tolerance,2)

beat_length_estimations = np.diff(chorus_beat_positions)
length_deviations = np.abs(rounded_beat_length - beat_length_estimations)

large_deviation_indices = np.where(length_deviations > 0.011)[0]

print(large_deviation_indices)

In [ ]:
track = load_track(title, fs)[0]

drop_beat_idx, possible_drops_beat_indices = drop_detection(track, beat_positions, fs, 2)

In [ ]:
chorus_beat_positions = beat_positions[drop_beat_idx:drop_beat_idx+17]
check_chorus_beat_grid(chorus_beat_positions, beat_length, 64)

## Plot

In [ ]:
energy_levels(title, bar_energies, possible_drops, estimated_drop, 
                   epsilon=eps ,save=False, plot_title='')

In [ ]:
plot_energy_levels(title, smoothed_cell_energies, possible_drops, estimated_drop, 
                   epsilon=eps ,save=False, plot_title='')

# Batch Processing

In [ ]:
#for i in tqdm(range(len(track_titles))):
for title in bad_examples:
    
    try:
        print(title)
        #title = track_titles[i]
        BPM = float(track_dicts[title]['BPM'])
        beat_length = 60/BPM
        fs = 44100
        
        #dpqb = 0.0
        #if title in bad_examples:
        #    with open(os.path.join('../data/bassline_extraction/beat_grid/bad_examples',title+'.txt'), 'r') as infile:
        #        dpqb = float(infile.read())
        #    print("Beat grid problematic! dpqb: {:.2f}%\n{}".format(dpqb,title))
        
        #if dpqb > 15:
        #    continue
        
        """        beat_positions = get_beat_positions(title)
        bar_positions = get_bar_positions(beat_positions)

        track = load_track(title, fs)[0]

        track_lp = lp_and_normalize(256, fs, 1001, track)

        bar_energies = calculate_bar_energies(track_lp, bar_positions, fs)
        cell_energies = calculate_cell_energies(bar_energies)
        smoothed_cell_energies = sample_and_hold(cell_energies, 4)

        eps = 2
        possible_drops = find_drops(smoothed_cell_energies, epsilon=eps)
        estimated_drop = drop_picking(possible_drops)
        
        drop_beat_idx = estimated_drop[0]*4"""

        chorus_beat_positions = beat_positions[drop_beat_idx:drop_beat_idx+16]
        print('Beat Positions')
        print(chorus_beat_positions)
        print('Beat Lengths')
        print(np.diff(chorus_beat_positions))
        
        large_deviation_indices = check_chorus_beat_grid(chorus_beat_positions, beat_length, 64)
        print(large_deviation_indices)
        print('\n')

        #plot_energy_levels(title, bar_energies, possible_drops, estimated_drop, 
        #                   show=False, epsilon=eps ,save=True)
        
    except KeyError:
        print("Track doesn't exist in track_dicts: {}\n".format(title))        
    except Exception as ex:     
        print("There was an error on: {}".format(title))
        exception_str = ''.join(traceback.format_exception(etype=type(ex), value=ex, tb=ex.__traceback__))
        print(exception_str+'\n')       